# EP1 - Conjunto de Maldelbrot e paralelização com pThreads e OpenMP

| Nome | NUSP |
|------|------|
| Giulia C. de Nardi | 10692203 |
| Vitor D. Tamae | 10705620 |
| Lucy Anne de Omena Evangelista | 11221776 |
| Leonardo Costa Santos | 10783142 |
| Alexandre Muller Jones | 8038149 |

Atualizando os pacotes Julia:

In [1]:
] up

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


Verificando o status dos pacotes, e se há algum problema, com o comando:

In [5]:
] st

    Status `~/.julia/environments/v1.3/Project.toml`
  [336ed68f] CSV v0.6.2
  [a93c6f00] DataFrames v0.21.0
  [31c24e10] Distributions v0.23.2
  [7073ff75] IJulia v1.21.2
  [b964fa9f] LaTeXStrings v1.1.0
  [8314cec4] PGFPlotsX v1.2.6
  [1a8c2f83] Query v0.12.2
  [f3b207a7] StatsPlots v0.14.6


## Mandelbrot sequencial

Nesta parte, traremos a execução do maldelbrot em sua versão sequencial, junto com a análise de tempo de execução para os diferentes tipo de fractais Triple Spiral, Elephant, Seahorse & Full) e em diferentes resoluções ($ 2^4 \cdots 2^13$)

In [6]:
; make mandelbrot_seq

make: 'mandelbrot_seq' is up to date.


In [7]:
; ./mandelbrot_seq

usage: ./mandelbrot_seq c_x_min c_x_max c_y_min c_y_max image_size
examples with image_size = 11500:
    Full Picture:         ./mandelbrot_seq -2.5 1.5 -2.0 2.0 11500
    Seahorse Valley:      ./mandelbrot_seq -0.8 -0.7 0.05 0.15 11500
    Elephant Valley:      ./mandelbrot_seq 0.175 0.375 -0.1 0.1 11500
    Triple Spiral Valley: ./mandelbrot_seq -0.188 -0.012 0.554 0.754 11500


In [17]:
; ./mandelbrot_seq 0.175 0.375 -0.1 0.1 200 0

A função abaixo recebe parâmetros `size`, com o tamanho da imagem, `f`, com a id do tupo de fractal a ser feito (0 - Full, 2 - Triple Spiral, 3 - Elephant, 4 - Seahorse), `mandel`, com o nome da função a ser executada (`./mandelbrot_seq`, `./mandelbrot_opm`, `./mandelbrot_pth`),e `threads`, com o número de threads do programa paralelo. A função executa o programa `mandelbrot` com os parâmetros dados e devolve um `DataFrame` com os resultados.

**#precisamos colocar a duração do programa como uma saida da funcao pra conseguir fazer as estimativas.**

**#tudo feito daqui pra baixo leva em conta que a duração do programa é uma das saídas**

In [14]:
using DataFrames, Query, StatsPlots, Statistics

function measure_mandelbrot(size, f, mandel; threads = 0)    
    if f == 0  mode = "-2.5 1.5 -2.0 2.0"
    elseif f == 1  mode = "-0.188 -0.012 0.554 0.754"
    elseif f == 2  mode = "0.175 0.375 -0.1 0.1"
    elseif f == 3  mode = "-0.8 -0.7 0.05 0.15"
    end
    
    results = parse.(Float64,
        split(chomp(read("./$mandel $mode $size $threads", String)), ","))
        
    return DataFrame(size = size,
        f = f,
        threads = threads,
        duration = results[1])
end

measure_mandelbrot (generic function with 1 method)

A função `run_experiments` recebe os mesmos parâmetros `size`, `f`,`mandel` e `threads`, e um parâmetro adicional `repetitions`, com o número de repetições de cada experimento com um dado número de `threads`. A função devolve um `DataFrame` com todos os experimentos.

In [16]:
function run_experiments(size, f, mandel, repetitions; threads = 0)
    run(`make $mandel`)
    
    results = DataFrame(size = Int[],
        f = Int[],
        threads = Int[],
        duration = Float64[]) 
    
    if threads != 0
    for t in threads
        for r in 1:repetitions
            append!(results,
                measure_mandel(size, f, mandel))    
        end
    end
    else
       for r in 1:repetitions
            append!(results,
                measure_mandel(size, f, mandel, t))    
        end 
    end 
    
    return results
end

run_experiments (generic function with 1 method)